# Studi Kasus Kedua: Sentiment Analysis

Di modul kali ini Anda akan mempelajari analisis sentimen media sosial menggunakan data teks dalam Bahasa Indonesia, yaitu dataset IndoNLU. Hingga akhir modul ini, diharapkan Anda dapat:

- Memahami apa itu analisis sentimen
- Memahami contoh penerapan Natural Language Processing
- Memahami data pipeline kasus Natural Language Processing
- Memahami teknik feature engineering untuk representasi teks
- Melakukan analisis sentimen dengan teknik Deep Learning
- Melakukan analisis sentimen dengan algoritma Support Vector Machine

---

In [2]:
%pip install --upgrade torch torchvision
%pip install --upgrade transformers

  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/19/b8/9f9f6b40d6b485f42ef560990e27722046d3bcd0ebcde47d54adc2d74432/torch-2.3.1-cp39-cp39-win_amd64.whl.metadata
  Using cached torch-2.3.1-cp39-cp39-win_amd64.whl.metadata (26 kB)
  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/3f/73/5dadc116a8e6115f0f3c015ed0c415d301330ddb385bc6c501686e019443/torchvision-0.18.1-cp39-cp39-win_amd64.whl.metadata
  Using cached torchvision-0.18.1-cp39-cp39-win_amd64.whl.metadata (6.6 kB)
  Obtaining dependency information for sympy from https://files.pythonhosted.org/packages/61/53/e18c8c97d0b2724d85c9830477e3ebea3acf1dcdc6deb344d5d9c93a9946/sympy-1.12.1-py3-none-any.whl.metadata
  Using cached sympy-1.12.1-py3-none-any.whl.metadata (12 kB)
  Obtaining dependency information for networkx from https://files.pythonhosted.org/packages/d5/f0/8fbc882ca80cf077f1b246c0e3c3465f7f415439bdea6b899f6b19f61f70/networkx-3.2.1-py3-no

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'D:\\AI-ML_Project\\machine-learning-terapan\\.venv\\Lib\\site-packages\\torch\\lib\\dnnl.lib'
Check the permissions.


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!git clone https://github.com/indobenchmark/indonlu

Cloning into 'indonlu'...


Perhatikan folder indonlu > dataset > smsa_doc_sentiment_prosa. Pada folder itulah data kita berada. Terlihat beberapa file seperti train_preprocess.tsv (ini adalah data latih yang akan kita gunakan), dan valid_preprocess.tsv (ini adalah data validasi yang akan kita gunakan). 

In [5]:
import random
import numpy as np
import pandas as pd
import torch
from torch import optim
import torch.nn.functional as F
from tqdm import tqdm
 
from transformers import BertForSequenceClassification, BertConfig, BertTokenizer
from nltk.tokenize import TweetTokenizer
 
from indonlu.utils.forward_fn import forward_sequence_classification
from indonlu.utils.metrics import document_sentiment_metrics_fn
from indonlu.utils.data_utils import DocumentSentimentDataset, DocumentSentimentDataLoader

In [6]:
###
# common functions
###
def set_seed(seed):
    # Mengatur dan menetapkan random seed.
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
def count_param(module, trainable=False):
    # Menghitung jumlah parameter dalam model
    if trainable:
        return sum(p.numel() for p in module.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in module.parameters())
    
def get_lr(optimizer):
    # Mengatur learning rate
    for param_group in optimizer.param_groups:
        return param_group['lr']
 
def metrics_to_string(metric_dict):
    # Mengonversi metriks ke dalam string
    string_list = []
    for key, value in metric_dict.items():
        string_list.append('{}:{:.2f}'.format(key, value))
    return ' '.join(string_list)

In [7]:
# Set random seed
set_seed(19072021)

Anda bebas mengatur random seed asalkan dalam bentuk angka atau integer. Tujuan mengatur random seed adalah agar model memberikan hasil yang sama setiap kali kita melakukan proses training. Untuk memudahkan, set random_seed dengan tanggal saat kita menjalankan kode tersebut.

## Konfigurasi dan Load Pre-trained Model

Tahap selanjutnya adalah proses Load Model dan konfigurasi. Pada tahap ini, kita menggunakan pre-trained model Indobert-base-p1 yang memiliki 124.5 juta parameter. Untuk tipe Indobert lainnya, Anda dapat melihatnya pada tautan berikut. 

Model Indobert dibangun berdasarkan general-purpose architecture BERT (Bidirectional Encoder Representation from Transformers).  BERT didesain untuk membantu komputer memahami arti bahasa ambigu dalam teks. Caranya adalah menggunakan teks di sekitarnya untuk membangun konteks.

In [8]:
# Load Tokenizer and Config
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1')
config = BertConfig.from_pretrained('indobenchmark/indobert-base-p1')
config.num_labels = DocumentSentimentDataset.NUM_LABELS
 
# Instantiate model
model = BertForSequenceClassification.from_pretrained('indobenchmark/indobert-base-p1', config=config)

tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

D:\AI-ML_Project\machine-learning-terapan\.venv\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Asus\.cache\huggingface\hub\models--indobenchmark--indobert-base-p1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/229k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

D:\AI-ML_Project\machine-learning-terapan\.venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [10]:
count_param(model)

124443651

## Persiapan Dataset Analisis Sentimen

In [24]:
train_dataset_path = 'D:/AI-ML_Project/machine-learning-terapan/learnings/indonlu/dataset/smsa_doc-sentiment-prosa/train_preprocess.tsv'
valid_dataset_path = 'D:/AI-ML_Project/machine-learning-terapan/learnings/indonlu/dataset/smsa_doc-sentiment-prosa/valid_preprocess.tsv'
test_dataset_path = 'D:/AI-ML_Project/machine-learning-terapan/learnings/indonlu/dataset/smsa_doc-sentiment-prosa/test_preprocess_masked_label.tsv'

Setelah menentukan lokasi dataset, kita perlu menyiapkan data dengan Pytorch. PyTorch menyediakan cara terstandarisasi untuk menyiapkan data sebelum melakukan pemodelan. PyTorch menyediakan banyak fitur canggih untuk memproses data. 

Di sini, kita akan menggunakan 2 kelas yang disediakan di PyTorch dalam modul torch.utils.data yaitu Dataset dan DataLoader. Disadur dari Pre-Trained Models for NLP Tasks Using PyTorch [39], kelas Dataset adalah sebuah abstract class yang perlu kita extend di PyTorch. Sedangkan, DataLoader adalah inti dari perangkat pemrosesan data di PyTorch. DataLoader menyediakan banyak fungsionalitas untuk mempersiapkan data termasuk berbagai metode sampling, komputasi paralel, dan pemrosesan terdistribusi. Nah, kita akan memindahkan objek dari kelas Dataset ke dalam objek dari kelas DataLoader untuk pemrosesan batch data lebih lanjut.

Untuk menunjukan bagaimana cara mengimplementasikan Dataset dan DataLoader di PyTorch, kita akan melihat lebih dalam pada kelas DocumentSentimentDataset dan DocumentSentimentDataLoader yang disediakan oleh IndoNLU. Anda dapat mengaksesnya di tautan berikut: data_utils.py.

Selanjutnya, kita akan mengimplementasikan kelas DocumentSentimentDataset untuk data loading. Untuk membuat kelas DocumentSentimentDataset yang fungsional, implementasikan 3 fungsi berikut, __init__(self, ...), __getitem__(self, index), dan __len__(self). 

Kemudian, implementasikan juga fungsi __getitem__(self, index) dan __len__(self). Sehingga, definisi lengkap dari kelas DocumentSentimentDataset adalah sebagai berikut.

In [14]:
from torch.utils.data import Dataset


class DocumentSentimentDataset(Dataset):
    # Static constant variable
    LABEL2INDEX = {'positive': 0, 'neutral': 1, 'negative': 2} # Map dari label string ke index
    INDEX2LABEL = {0: 'positive', 1: 'neutral', 2: 'negative'} # Map dari Index ke label string
    NUM_LABELS = 3 # Jumlah label
   
    def load_dataset(self, path):
        df = pd.read_csv(path, sep='\t', header=None) # Baca tsv file dengan pandas
        df.columns = ['text','sentiment'] # Berikan nama pada kolom tabel
        df['sentiment'] = df['sentiment'].apply(lambda lab: self.LABEL2INDEX[lab]) # Konversi string label ke index
        return df
   
    def __init__(self, dataset_path, tokenizer, *args, **kwargs):
        self.data = self.load_dataset(dataset_path) # Load tsv file
 
        # Assign tokenizer, disini kita menggunakan tokenizer subword dari HuggingFace
        self.tokenizer = tokenizer 
 
    def __getitem__(self, index):
        data = self.data.loc[index,:] # Ambil data pada baris tertentu dari tabel
        text, sentiment = data['text'], data['sentiment'] # Ambil nilai text dan sentiment
        subwords = self.tokenizer.encode(text) # Tokenisasi text menjadi subword
    
    # Return numpy array dari subwords dan label
        return np.array(subwords), np.array(sentiment), data['text']
   
    def __len__(self):
        return len(self.data)  # Return panjang dari dataset

In [16]:
from torch.utils.data import DataLoader


class DocumentSentimentDataLoader(DataLoader):
    def __init__(self, max_seq_len=512, *args, **kwargs):
        super(DocumentSentimentDataLoader, self).__init__(*args, **kwargs)
        self.max_seq_len = max_seq_len # Assign batas maksimum subword
        self.collate_fn = self._collate_fn # Assign fungsi collate_fn dengan fungsi yang kita definisikan
       
    def _collate_fn(self, batch):
        batch_size = len(batch) # Ambil batch size
        max_seq_len = max(map(lambda x: len(x[0]), batch)) # Cari panjang subword maksimal dari batch 
        max_seq_len = min(self.max_seq_len, max_seq_len) # Bandingkan dengan batas yang kita tentukan sebelumnya
       
    # Buat buffer untuk subword, mask, dan sentiment labels, inisialisasikan semuanya dengan 0
        subword_batch = np.zeros((batch_size, max_seq_len), dtype=np.int64)
        mask_batch = np.zeros((batch_size, max_seq_len), dtype=np.float32)
        sentiment_batch = np.zeros((batch_size, 1), dtype=np.int64)
       
    # Isi semua buffer
        for i, (subwords, sentiment, raw_seq) in enumerate(batch):
            subwords = subwords[:max_seq_len]
            subword_batch[i,:len(subwords)] = subwords
            mask_batch[i,:len(subwords)] = 1
            sentiment_batch[i,0] = sentiment
           
    # Return subword, mask, dan sentiment data
        return subword_batch, mask_batch, sentiment_batch

In [25]:
train_dataset = DocumentSentimentDataset(train_dataset_path, tokenizer, lowercase=True)
valid_dataset = DocumentSentimentDataset(valid_dataset_path, tokenizer, lowercase=True)
test_dataset = DocumentSentimentDataset(test_dataset_path, tokenizer, lowercase=True)
 
train_loader = DocumentSentimentDataLoader(dataset=train_dataset, max_seq_len=512, batch_size=32, num_workers=16, shuffle=True)  
valid_loader = DocumentSentimentDataLoader(dataset=valid_dataset, max_seq_len=512, batch_size=32, num_workers=16, shuffle=False)  
test_loader = DocumentSentimentDataLoader(dataset=test_dataset, max_seq_len=512, batch_size=32, num_workers=16, shuffle=False)

D:\AI-ML_Project\machine-learning-terapan\.venv\lib\site-packages\torch\utils\data\dataloader.py:558: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [26]:
print(train_dataset[0])

(array([    2,  6540,    92,  2970,   213,  4259,  3553,   899,    34,
         259,  5590,   262,  2558,   386,   899,  1687,    26,  1574,
       30470,   899,  3310, 30468, 22130, 30360,  6123,  6368, 30468,
       22130, 30360,  2652,  1746, 30468,  8869,  6540,    34,  6315,
        1622,  1256,  8949,   899, 30468,  4222,  1622,   752,   245,
         295,  2083, 30470,  2346,  7107,   300, 30470,   405,   724,
        5189, 30470,   843, 17464,   899,   540, 10989,  3331,  1107,
       30468,   119,  3221,    79,    34,  2170,    98,  9167, 30457,
           3]), array(0, dtype=int64), 'warung ini dimiliki oleh pengusaha pabrik tahu yang sudah puluhan tahun terkenal membuat tahu putih di bandung . tahu berkualitas , dipadu keahlian memasak , dipadu kretivitas , jadilah warung yang menyajikan menu utama berbahan tahu , ditambah menu umum lain seperti ayam . semuanya selera indonesia . harga cukup terjangkau . jangan lewatkan tahu bletoka nya , tidak kalah dengan yang asli dari te

In [27]:
w2i, i2w = DocumentSentimentDataset.LABEL2INDEX, DocumentSentimentDataset.INDEX2LABEL
print(w2i)
print(i2w)

{'positive': 0, 'neutral': 1, 'negative': 2}
{0: 'positive', 1: 'neutral', 2: 'negative'}


## Uji Model dengan Contoh Kalimat

In [28]:
text = 'Bahagia hatiku melihat pernikahan putri sulungku yang cantik jelita'
subwords = tokenizer.encode(text)
subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)
 
logits = model(subwords)[0]
label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()
 
print(f'Text: {text} | Label : {i2w[label]} ({F.softmax(logits, dim=-1).squeeze()[label] * 100:.3f}%)')

Text: Bahagia hatiku melihat pernikahan putri sulungku yang cantik jelita | Label : positive (52.480%)


## Fine Tuning dan Evaluasi

In [30]:
optimizer = optim.Adam(model.parameters(), lr=3e-6)
# model = model.cuda()

Selanjutnya, kita akan melatih model dengan jumlah epoch = 5. Tahapan yang akan kita lakukan, antara lain:

- Proses pelatihan dan pembaruan model
- Kalkulasi metriks pelatihan
- Evaluasi pada data validasi 
- Kalkulasi matriks validasi

In [ ]:
# Train
n_epochs = 5
for epoch in range(n_epochs):
    model.train()
    torch.set_grad_enabled(True)
 
    total_train_loss = 0
    list_hyp, list_label = [], []
 
    train_pbar = tqdm(train_loader, leave=True, total=len(train_loader))
    for i, batch_data in enumerate(train_pbar):
        # Forward model
        loss, batch_hyp, batch_label = forward_sequence_classification(model, batch_data[:-1], i2w=i2w, device='cuda')
 
        # Update model
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
 
        tr_loss = loss.item()
        total_train_loss = total_train_loss + tr_loss
 
        # Calculate metrics
        list_hyp += batch_hyp
        list_label += batch_label
 
        train_pbar.set_description("(Epoch {}) TRAIN LOSS:{:.4f} LR:{:.8f}".format((epoch+1),
            total_train_loss/(i+1), get_lr(optimizer)))
 
    # Calculate train metric
    metrics = document_sentiment_metrics_fn(list_hyp, list_label)
    print("(Epoch {}) TRAIN LOSS:{:.4f} {} LR:{:.8f}".format((epoch+1),
        total_train_loss/(i+1), metrics_to_string(metrics), get_lr(optimizer)))
 
    # Evaluate on validation
    model.eval()
    torch.set_grad_enabled(False)
    
    total_loss, total_correct, total_labels = 0, 0, 0
    list_hyp, list_label = [], []
 
    pbar = tqdm(valid_loader, leave=True, total=len(valid_loader))
    for i, batch_data in enumerate(pbar):
        batch_seq = batch_data[-1]        
        loss, batch_hyp, batch_label = forward_sequence_classification(model, batch_data[:-1], i2w=i2w, device='cuda')
        
        # Calculate total loss
        valid_loss = loss.item()
        total_loss = total_loss + valid_loss
 
        # Calculate evaluation metrics
        list_hyp += batch_hyp
        list_label += batch_label
        metrics = document_sentiment_metrics_fn(list_hyp, list_label)
 
        pbar.set_description("VALID LOSS:{:.4f} {}".format(total_loss/(i+1), metrics_to_string(metrics)))
        
    metrics = document_sentiment_metrics_fn(list_hyp, list_label)
    print("(Epoch {}) VALID LOSS:{:.4f} {}".format((epoch+1),
        total_loss/(i+1), metrics_to_string(metrics)))

  0%|          | 0/344 [00:00<?, ?it/s]